# 文件操作

## 文件基本概念

文件就是存放在硬盘上的一坨数据，可以是文本、图片、视频、代码等，跟程序运行时待在内存里的数据相对。

In [1]:
# 简单示例：把一段文字写进文件，再读出来
text = "你好，文件！"

# 写入磁盘
f = open("hello.txt", "w", encoding="utf-8")
f.write(text)
f.close()

# 再从磁盘读回来
f = open("hello.txt", "r", encoding="utf-8")
content = f.read()
f.close()

print(content)

你好，文件！


有了文件，你关掉程序、关掉电脑，数据也不会丢，这就是“持久化”的好处。

### 文本文件和二进制文件

文本文件可以看成“很长的字符串”，有统一编码（比如 UTF-8），而二进制文件就是原始的 0 和 1，没有“字符”的概念。

In [2]:
# 文本方式写入（默认）
with open("note.txt", "w", encoding="utf-8") as f:
    f.write("今天学了文件操作。")

# 二进制方式读取图片
with open("picture.jpg", "rb") as f:
    data = f.read()
    print("读取到的字节数：", len(data))

读取到的字节数： 178295


区分文本文件和二进制文件，可以让你正确地读写日志（文本）和图片/视频/音频（二进制）。

### 文件路径：相对路径和绝对路径

路径就是“怎么走到这个文件”的路线，可以写成“从当前文件夹出发”（相对路径）或者“从盘符根目录出发”（绝对路径）。

In [3]:
# 相对路径（从当前目录出发）
f1 = open("./data.txt", "w", encoding="utf-8")
f1.write("相对路径示例")
f1.close()

# 绝对路径（macOS 示例，注意根据自己电脑改路径）
# 假设你的用户名是 yourname，在桌面下的 Hello 目录里创建文件：
# f2 = open("/Users/yourname/Desktop/Hello/data.txt", "w", encoding="utf-8")
# f2.write("绝对路径示例")
# f2.close()


搞清楚路径的写法，才能让程序在正确的位置读到/写出文件，不会一脸懵地说“找不到文件”。

## 打开和关闭文件

### 使用 open 打开文件

用 `open()` 可以打开一个已经存在的文件，或者在需要写入时新建一个文件，它会给你一个“文件对象”来读写数据。

In [5]:
# 只读打开一个文件（文件必须已经存在）
f = open("hello.txt", "r", encoding="utf-8")
print("read 模式，文件名：", f.name)
f.close()

# 写入模式，没有就创建，有就清空重写
f = open("output.txt", "w", encoding="utf-8")
f.write("第一次写入内容")
f.close()

# 追加模式，在原内容后面接着写
f = open("output.txt", "a", encoding="utf-8")
f.write("\n第二次追加内容")
f.close()

read 模式，文件名： hello.txt


掌握几种最常见的打开模式，就能应付 90% 的文件读写需求。

> 常用模式简记（够用了）：
>
> * `"r"`：只读，文件必须存在
> * `"w"`：只写，文件不存在就创建，存在就清空再写
> * `"a"`：追加写入，在后面加内容
> * 加上 `"b"`：二进制模式，比如 `"rb"`、`"wb"`

### 用 with 自动关闭文件

手动 `close()` 容易忘，用 `with` 可以在读写结束后自动帮你关文件，出错也没关系。

In [6]:
with open("note.txt", "w", encoding="utf-8") as f:
    f.write("使用 with 打开的文件，会自动关闭。")

# 此时不需要 f.close()
print("代码执行完，文件已经被自动关闭。")

代码执行完，文件已经被自动关闭。


习惯用 `with open(...) as f` 写文件操作，可以大大减少忘记关闭文件带来的各种奇怪问题。

## 文件读写

### 把内容写入文件：write

写文件最常见的方式就是用 `write()` 把字符串写进去，它不会自动换行，需要你自己加 `\n`。

In [7]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("hello world\n")
    f.write("nihao python\n")

print("写入完成！")

写入完成！


写文件的核心就是把内存里的字符串落盘，之后你就可以关掉程序，下次再打开继续用。

### 一次性读完整个文件

`read()` 会把文件内容一次性全部读出来，适合体积不大的文本文件。

In [8]:
# 先写一点内容
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("hello world\n")
    f.write("nihao python\n")

# 读取全部内容
with open("test.txt", "r", encoding="utf-8") as f:
    content = f.read()

print("文件内容是：")
print(content)

文件内容是：
hello world
nihao python



当文件不大时，用 `read()` 一口气读完是最省事的方式。

### 按指定长度读取

给 `read()` 传入 `size`，就可以每次只读一部分内容（按字节/字符计），适合“边读边处理”。

In [9]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("abcdefg1234567")

with open("test.txt", "r", encoding="utf-8") as f:
    part1 = f.read(5)
    part2 = f.read(4)
    rest = f.read()

print("第一次读取：", part1)
print("第二次读取：", part2)
print("剩余内容：", rest)

第一次读取： abcde
第二次读取： fg12
剩余内容： 34567


分段读取可以在文件很大、但你每次只处理一小块的时候用，避免一次性把大文件全部塞进内存。

### 按行读取

`readline()` 每次读取一行（包含结尾的换行符），适合一行一行地处理日志、配置等文本。

In [10]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("第一行\n")
    f.write("第二行\n")
    f.write("第三行\n")

with open("test.txt", "r", encoding="utf-8") as f:
    line1 = f.readline()
    line2 = f.readline()

print("line1：", repr(line1))
print("line2：", repr(line2))

line1： '第一行\n'
line2： '第二行\n'


一行一行读，可以在发现某一行符合条件时立刻处理，而不用等把整个文件读完。

### 一次读出所有行

`readlines()` 会把文件中的每一行放到列表里，每个元素是一行的字符串。

In [11]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("行1\n行2\n行3\n")

with open("test.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

print("列表长度：", len(lines))
print("第一行：", repr(lines[0]))
print("所有行：", lines)

列表长度： 3
第一行： '行1\n'
所有行： ['行1\n', '行2\n', '行3\n']


当你需要“多次遍历这些行”或者“随机访问某一行”时，把所有行先读到列表里会比较方便。

## 文件位置与随机访问

### 查看和移动“读写光标”：tell 和 seek

文件内部有一个“光标”，表示当前读写到哪里了，`tell()` 可以看位置，`seek()` 可以移动位置。

In [12]:
with open("sample.txt", "w", encoding="utf-8") as f:
    f.write("ABCDEFG")

with open("sample.txt", "r", encoding="utf-8") as f:
    print("初始位置：", f.tell())
    print("读三个字符：", f.read(3))
    print("当前位置：", f.tell())

    # 把光标移动回文件开头
    f.seek(0)
    print("重新回到开头：", f.tell())
    print("再次读取：", f.read(2))

初始位置： 0
读三个字符： ABC
当前位置： 3
重新回到开头： 0
再次读取： AB


当你只想修改/读取文件中间的一小段内容时，`seek()` 可以让你精确跳转到那一段。

> 注意：
>
> * 对于二进制模式（`"rb"` / `"wb"`）下的 `seek()`，偏移量是按字节算的。
> * 在很多系统中，从末尾往前移动（`seek(..., 2)`）必须用二进制模式打开文件。

### 截断文件

`truncate(size)` 用来截断文件，只保留前面 `size` 个字节/字符（文本模式下按字符理解会更直观一点）。

In [13]:
with open("truncate_demo.txt", "w", encoding="utf-8") as f:
    f.write("这是一个很长很长的句子。")

# 截断，只保留前 6 个字符
with open("truncate_demo.txt", "r+", encoding="utf-8") as f:
    f.truncate(6)

with open("truncate_demo.txt", "r", encoding="utf-8") as f:
    print("截断后的内容：", f.read())

截断后的内容： 这是


当你想“砍掉文件后半部分”的时候，用 `truncate()` 会比重新写一个新文件省事。

## 操作文件和文件夹（os 模块）

下面这些函数来自 `os` 和 `os.path`，是日常操作文件和目录时最常用的一小撮。

### 创建、重命名、删除文件 / 目录

In [14]:
import os

# 当前工作路径
print("当前路径：", os.getcwd())

# 创建目录（单层）
os.mkdir("demo_dir")

# 在目录里创建一个文件
with open("demo_dir/hello.txt", "w", encoding="utf-8") as f:
    f.write("hello")

# 重命名文件
os.rename("demo_dir/hello.txt", "demo_dir/hi.txt")

# 删除文件
os.remove("demo_dir/hi.txt")

# 删除空目录
os.rmdir("demo_dir")

print("操作完成。")

当前路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics
操作完成。


用 `os` 操作文件和目录，可以让你的程序真正“动手整理硬盘”，而不是只在内存里打转。

### 递归创建和列出目录：makedirs 和 listdir

In [15]:
import os

# 递归创建多级目录
os.makedirs("a/b/c", exist_ok=True)

# 查看某个目录下的内容
print("当前目录下的内容：", os.listdir("."))

# 查看 a 目录下的内容
print("a 目录下的内容：", os.listdir("a"))

当前目录下的内容： ['01-basics.ipynb', '.DS_Store', 'note.txt', 'truncate_demo.txt', 'a', 'README.md', '05-file-operations.ipynb', '02-control-structures.ipynb', 'test.txt', '04-functions.ipynb', '03-container.ipynb', 'hello.txt', 'picture.jpg', 'output.txt', 'data.txt', 'sample.txt']
a 目录下的内容： ['b']


当你需要先确保某个深层目录存在，再往里面写文件时，`os.makedirs()` 特别方便。

### 路径判断与拼接：os.path

`os.path` 里有很多“关于路径的小工具”，最常用的是判断路径存在与否，以及一个“优雅的”拼接路径函数。

In [16]:
import os

# 拼接路径（自动处理 / 或 \）
base_dir = os.getcwd()
file_path = os.path.join(base_dir, "data", "log.txt")
print("拼接后的路径：", file_path)

# 判断路径是否存在
print("路径是否存在：", os.path.exists(file_path))

# 判断是文件还是目录
print("是文件吗？", os.path.isfile(file_path))
print("是目录吗？", os.path.isdir(os.path.dirname(file_path)))

拼接后的路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/data/log.txt
路径是否存在： False
是文件吗？ False
是目录吗？ False


用 `os.path.join()` 代替手写字符串拼接，可以避免因为不同系统分隔符不一样而导致路径错误。

## 递归遍历目录：os.walk

### 一次性拿到所有子目录和文件

`os.walk()` 会帮你把一个目录下面所有的子目录和文件统统扫一遍，一层一层地交给你处理。

In [17]:
import os

root_dir = os.getcwd()  # 从当前目录开始

for root, dirs, files in os.walk(root_dir):
    print("当前路径：", root)
    print("子目录列表：", dirs)
    print("文件列表：", files)
    print("-" * 30)

当前路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics
子目录列表： ['a']
文件列表： ['01-basics.ipynb', '.DS_Store', 'note.txt', 'truncate_demo.txt', 'README.md', '05-file-operations.ipynb', '02-control-structures.ipynb', 'test.txt', '04-functions.ipynb', '03-container.ipynb', 'hello.txt', 'picture.jpg', 'output.txt', 'data.txt', 'sample.txt']
------------------------------
当前路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/a
子目录列表： ['b']
文件列表： []
------------------------------
当前路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/a/b
子目录列表： ['c']
文件列表： []
------------------------------
当前路径： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/a/b/c
子目录列表： []
文件列表： []
------------------------------


当你需要对一个大项目里所有 `.py` 文件、所有图片、所有日志做统计或批量处理时，`os.walk()` 是最顺手的工具。

## 案例：拷贝文件

### 简单版：一次性读完再写入

In [18]:
def copy_file(src_path, dst_path):
    # 以二进制模式打开源文件和目标文件
    with open(src_path, "rb") as src:
        data = src.read()          # 一次性读完
    with open(dst_path, "wb") as dst:
        dst.write(data)            # 一次性写入

    print("复制完成：", src_path, "->", dst_path)


# 示例（请把路径改成自己电脑上真实存在的文件）
copy_file("/Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture.jpg", "/Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture2.jpg")

复制完成： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture.jpg -> /Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture2.jpg


这种写法最简单直观，适合小文件的复制。

### 优化版：分块读取，节省内存

对于大文件，一次性读完可能会占用很多内存，可以改成“读一点、写一点”的方式。

In [19]:
def copy_file_chunked(src_path, dst_path, chunk_size=1024):
    with open(src_path, "rb") as src, open(dst_path, "wb") as dst:
        while True:
            chunk = src.read(chunk_size)  # 读一块
            if not chunk:                 # 读完了就退出循环
                break
            dst.write(chunk)              # 写一块到目标文件
    print("分块复制完成：", src_path, "->", dst_path)


# 示例（同样需要把路径改成自己电脑上的）
copy_file("/Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture.jpg", "/Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture3.jpg")

复制完成： /Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture.jpg -> /Users/shenjiuyang/workspace/llm-learning/01-python-basics/picture3.jpg


这种分块复制方式非常适合大文件，既能稳定运行，又不会一下子吃掉太多内存。